In [2]:
import tensorflow as tf
from tensorflow import keras

# SCA koriscenjem deep learninga

U radu je do sada bilo prikazano nekoliko nacina izvlacenja tajni iz hardvera. State of the art pristup su template napadi koji su se pokazali kao standard u ovoj oblasti istrazivanja sigurnosti sistema. Kod template napada je potrebno ozbiljno preprocesiranje ulaznih podataka, potrebno je dosta iskustva. DPA i CPA su dovoljno jednostavni za koriscenje i implementaciju, ali je i za njih potreban relativno veliki broj trace-va i ne pokazuju se toliko dobro sa sumovitim podacima. Deep learning nastoji da eliminise neke od navedenih problema. 

# Arhitekture neuronskih mreza koje ce biti koriscene

U radu koji opisuje ASCAD bazu podataka date su i neke smernice oko odabira odgovarajucih modela i izbora hiperparametara. Preprocesiranje podataka i redukcija dimenzionalnosti nije potrebna, jer se ona obavi u prvim slojevima mreze.

### MLP (multi-layered perceptron)

Obicna potpuno povezana neuronska mreza sa softmax aktivacijom - normalizuje izlaze

input sloj : 80.000 tacaka traceva
output sloj : 256 klasa za 256 mogucih vrednosti jednog bajta kljuca


### ResNet

In [2]:
EXECUTE_IN_COLAB = True

if EXECUTE_IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
import os

def load_shards(path, num_shards=256):
  shard_array = []
  for shard_idx, shard_name in enumerate(os.listdir(path)):
    if shard_idx == num_shards:
      break
    shard_array.append(np.load(path + '/' + shard_name))

  return shard_array

In [9]:
training_data_path = '/content/drive/MyDrive/datasets/tinyaes/train'
testing_data_path = '/content/drive/MyDrive/datasets/tinyaes/test'

training_data = load_shards(training_data_path)
test_data = load_shards(testing_data_path)